<a href="https://colab.research.google.com/github/chiyanglin-AStar/2025_physics_note/blob/main/02_Pycuda_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is reference from [Linking Python to CUDA with PyCUDA: [PyCuda  Tutorial](https://documen.tician.de/pycuda/tutorial.html)

## PyCUDA ref:

[PyCUDA Tutorial(翻譯)](https://hackmd.io/@shaoeChen/SkbmZOXbB/https%3A%2F%2Fhackmd.io%2F%40shaoeChen%2FSkKb0fX-H)

[pycuda tutorial](https://documen.tician.de/pycuda/tutorial.html)

[PyCUDA Tutorial Introduction](https://github.com/berlinguyinca/pycuda/blob/master/doc/source/tutorial.rst)

[GPU程式設計(5) -- Python](https://ithelp.ithome.com.tw/articles/10283144)

In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.4/92.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=6aa0b44340df9123e1c544a0e72b09741afab15eb91d430af33022123d94d5ea
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


## Getting started

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

## Transferring Data

The next step in most programs is to transfer data onto the device. In PyCuda, you will mostly transfer data from numpy arrays on the host. (But indeed, everything that satisfies the Python buffer interface will work, even bytes.) Let’s make a 4x4 array of random numbers:

In [3]:
import numpy
a = numpy.random.randn(4,4)

But wait–a consists of double precision numbers, but most nVidia devices only support single precision:

In [4]:
a = a.astype(numpy.float32)

Finally, we need somewhere to transfer data to, so we need to allocate memory on the device:

In [5]:
a_gpu = cuda.mem_alloc(a.nbytes)

As a last step, we need to transfer the data to the GPU:

In [6]:
cuda.memcpy_htod(a_gpu, a)

## Executing a Kernel
For this tutorial, we’ll stick to something simple: We will write code to double each entry in a_gpu. To this end, we write the corresponding CUDA C code, and feed it into the constructor of a pycuda.compiler.SourceModule:

In [7]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

If there aren’t any errors, the code is now compiled and loaded onto the device. We find a reference to our pycuda.driver.Function and call it, specifying a_gpu as the argument, and a block size of 4x4:

In [8]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

Finally, we fetch the data back from the GPU and display it, together with the original a:

In [9]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

[[ 0.46608397 -1.261564   -0.05711028  3.14196   ]
 [-2.1014385  -5.111447   -0.59766185  1.3984003 ]
 [-0.1855822   0.04879789  0.6932547  -3.217177  ]
 [ 1.0169098   0.01087349 -0.40107808  2.1073337 ]]
[[ 0.23304199 -0.630782   -0.02855514  1.57098   ]
 [-1.0507193  -2.5557234  -0.29883093  0.69920015]
 [-0.0927911   0.02439895  0.34662735 -1.6085885 ]
 [ 0.5084549   0.00543674 -0.20053904  1.0536668 ]]


his will print something like this:
```
[[ 0.51360393  1.40589952  2.25009012  3.02563429]
 [-0.75841576 -1.18757617  2.72269917  3.12156057]
 [ 0.28826082 -2.92448163  1.21624792  2.86353827]
 [ 1.57651746  0.63500965  2.21570683 -0.44537592]]
[[ 0.25680196  0.70294976  1.12504506  1.51281714]
 [-0.37920788 -0.59378809  1.36134958  1.56078029]
 [ 0.14413041 -1.46224082  0.60812396  1.43176913]
 [ 0.78825873  0.31750482  1.10785341 -0.22268796]]
```

It worked! That completes our walkthrough. Thankfully, PyCuda takes over from here and does all the cleanup for you, so you’re done. Stick around for some bonus material in the next section, though.

(You can find the code for this demo as examples/demo.py in the PyCuda source distribution.)

## Shortcuts for Explicit Memory Copies
The pycuda.driver.In, pycuda.driver.Out, and pycuda.driver.InOut argument handlers can simplify some of the memory transfers. For example, instead of creating a_gpu, if replacing a is fine, the following code can be used:

In [10]:
func(cuda.InOut(a), block=(4, 4, 1))

## Prepared Invocations
Function invocation using the built-in pycuda.driver.Function.__call__() method incurs overhead for type identification (see Device Interface). To achieve the same effect as above without this overhead, the function is bound to argument types (as designated by Python’s standard library struct module), and then called. This also avoids having to assign explicit argument sizes using the numpy.number classes:

In [11]:
grid = (1, 1)
block = (4, 4, 1)
func.prepare("P")
func.prepared_call(grid, block, a_gpu)

## Bonus: Abstracting Away the Complications
Using a pycuda.gpuarray.GPUArray, the same effect can be achieved with much less writing:

In [12]:
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
import numpy

a_gpu = gpuarray.to_gpu(numpy.random.randn(4,4).astype(numpy.float32))
a_doubled = (2*a_gpu).get()
print(a_doubled)
print(a_gpu)

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()


[[-1.795199   -0.99280566  2.4260628   2.4114265 ]
 [-0.75836754  1.299085   -0.3551251   0.06969641]
 [ 1.0779378  -0.39955267  1.2220305   0.18321636]
 [-3.310843    0.09009165 -0.8409423   0.6559306 ]]
[[-0.8975995  -0.49640283  1.2130314   1.2057133 ]
 [-0.37918377  0.6495425  -0.17756255  0.03484821]
 [ 0.5389689  -0.19977634  0.61101526  0.09160818]
 [-1.6554215   0.04504583 -0.42047116  0.3279653 ]]


## Advanced Topics
### Structures
(contributed by Nicholas Tung, find the code in examples/demo_struct.py)

Suppose we have the following structure, for doubling a number of variable length arrays:

In [13]:
mod = SourceModule("""
    struct DoubleOperation {
        int datalen, __padding; // so 64-bit ptrs can be aligned
        float *ptr;
    };

    __global__ void double_array(DoubleOperation *a) {
        a = &a[blockIdx.x];
        for (int idx = threadIdx.x; idx < a->datalen; idx += blockDim.x) {
            a->ptr[idx] *= 2;
        }
    }
    """)

Each block in the grid (see CUDA documentation) will double one of the arrays. The for loop allows for more data elements than threads to be doubled, though is not efficient if one can guarantee that there will be a sufficient number of threads. Next, a wrapper class for the structure is created, and two arrays are instantiated:

In [14]:
class DoubleOpStruct:
    mem_size = 8 + numpy.intp(0).nbytes
    def __init__(self, array, struct_arr_ptr):
        self.data = cuda.to_device(array)
        self.shape, self.dtype = array.shape, array.dtype
        packed_args = struct.pack("ixP", array.size, numpy.uintp(self.data))
        cuda.memcpy_htod(struct_arr_ptr, packed_args)

    def __str__(self):
        return str(cuda.from_device(self.data, self.shape, self.dtype))

struct_arr = cuda.mem_alloc(2 * DoubleOpStruct.mem_size)
do2_ptr = int(struct_arr) + DoubleOpStruct.mem_size

array1 = DoubleOpStruct(numpy.array([1, 2, 3], dtype=numpy.float32), struct_arr)
array2 = DoubleOpStruct(numpy.array([0, 4], dtype=numpy.float32), do2_ptr)
print("original arrays", array1, array2)

NameError: name 'struct' is not defined